# Imports 

In [1]:
from common import getOrCreateSparkSession

In [2]:
spark = getOrCreateSparkSession()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5e9af8a9-ae41-4e03-8243-c00dbe3f0fbb;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 in central
:: resolution report :: resolve 109ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| sear

# Branch out 

In [4]:
spark.sql("CREATE BRANCH dev IN nessie FROM main").toPandas()

,refType,name,hash
0,Branch,dev,c7d26b422a1f7a0e6eac5ee670c3da8a5a6595e2e8e3e4...


In [5]:
spark.sql("LIST REFERENCES IN nessie").toPandas()

,refType,name,hash
0,Branch,dev,c7d26b422a1f7a0e6eac5ee670c3da8a5a6595e2e8e3e4...
1,Branch,main,c7d26b422a1f7a0e6eac5ee670c3da8a5a6595e2e8e3e4...


In [6]:
spark.sql("USE REFERENCE dev IN nessie").toPandas()

,refType,name,hash
0,Branch,dev,c7d26b422a1f7a0e6eac5ee670c3da8a5a6595e2e8e3e4...


In [7]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS nessie.taxis.vendors (
    vendor_id INT,
    title STRING
)
USING iceberg;
"""
).toPandas()

""


In [8]:
spark.sql("SHOW TABLES IN nessie;").toPandas()

,namespace,tableName,isTemporary
0,taxis,trips,False
1,taxis,vendors,False


In [9]:
spark.sql(
"""
INSERT INTO nessie.taxis.vendors VALUES
    (1, "Yellow Taxi"),
    (10, "We'll Get You There Compnay"),
    (4252, "Not Uber")
"""
).toPandas()

""


In [10]:
spark.sql("SELECT COUNT(*) AS total_vendors FROM nessie.taxis.vendors;").toPandas()

,total_vendors
0,3


In [11]:
spark.sql("SELECT COUNT(*) AS total_trips FROM nessie.taxis.trips;").toPandas()

,total_trips
0,15


In [12]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW new_trips USING csv
OPTIONS (path "/data/yellow_tripdata_sample_2019_02.csv", header true);
"""
).toPandas()

""


In [13]:
spark.sql("""
INSERT INTO nessie.taxis.trips
SELECT 
    CAST(vendor_id AS INT),
    CAST(pickup_datetime AS TIMESTAMP),
    CAST(dropoff_datetime AS TIMESTAMP),
    CAST(passenger_count AS INT),
    CAST(pickup_location_id AS INT),
    CAST(dropoff_location_id AS INT),
    CAST(fare_amount AS FLOAT)
FROM new_trips;
"""
).toPandas()

""


In [14]:
spark.sql("SELECT COUNT(*) FROM nessie.taxis.trips;").toPandas()

,count(1)
0,20


In [16]:
spark.sql("SELECT COUNT(*) FROM new_trips;").toPandas()

,count(1)
0,5


In [17]:
spark.sql("DROP TABLE new_trips;").toPandas()

""


# Merge back into `@main`

In [18]:
spark.sql("SHOW TABLES IN nessie").toPandas()

,namespace,tableName,isTemporary
0,taxis,trips,False
1,taxis,vendors,False


In [20]:
spark.sql("MERGE BRANCH dev INTO main IN nessie").toPandas()

,name,hash
0,main,1de998576faf154bf496cb3262ac45472e406eac2bb2a1...


In [21]:
spark.sql("USE REFERENCE main IN nessie").toPandas()

,refType,name,hash
0,Branch,main,1de998576faf154bf496cb3262ac45472e406eac2bb2a1...


In [22]:
spark.sql("SHOW TABLES IN nessie").toPandas()

,namespace,tableName,isTemporary
0,taxis,trips,False
1,taxis,vendors,False


In [23]:
spark.sql("SELECT COUNT(*) FROM nessie.taxis.trips").toPandas()

,count(1)
0,20


# Clean Up

In [25]:
spark.sql("DROP BRANCH IF EXISTS dev IN nessie").toPandas()

,status
0,OK
